<a href="https://colab.research.google.com/github/kefasmanu/NLP-FELLOWSHIP/blob/main/week%206/%20day%204/%20nlpnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
# import os
# os.chdir("/content/gdrive/MyDrive/NLP_Fellowship_2022/Notebooks/Week 4/data/")
# !ls

In [3]:
# !rm rf NLP-FELLOWSHIP/

In [4]:
!git clone https://github.com/agent87/NLP-FELLOWSHIP.git

Cloning into 'NLP-FELLOWSHIP'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 91 (delta 10), reused 41 (delta 10), pack-reused 50
Unpacking objects: 100% (91/91), done.


In [5]:
import pandas as pd

full_dataset = pd.read_csv('/content/NLP-FELLOWSHIP/Data/50k_imdb_movie_reviews.csv')
full_dataset.head()

,review,sentiment,set
0,I went and saw this movie last night after bei...,1,test
1,Actor turned director Bill Paxton follows up h...,1,test
2,As a recreational golfer with some knowledge o...,1,test
3,"I saw this film in a sneak preview, and it is ...",1,test
4,Bill Paxton has taken the true story of the 19...,1,test


In [6]:
full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'train')]     

,review,sentiment,set
25000,Bromwell High is a cartoon comedy. It ran at t...,1,train
25001,Homelessness (or Houselessness as George Carli...,1,train
25002,Brilliant over-acting by Lesley Ann Warren. Be...,1,train
25003,This is easily the most underrated film inn th...,1,train
25004,This is not the typical Mel Brooks film. It wa...,1,train
...,...,...,...
37495,"Seeing as the vote average was pretty low, and...",1,train
37496,"The plot had some wretched, unbelievable twist...",1,train
37497,I am amazed at how this movie(and most others ...,1,train
37498,A Christmas Together actually came before my t...,1,train


In [7]:
print("No of positive in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'train')])))
print("No of negative in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'train')])))
print("No of positive in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'test')])))
print("No of negative in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'test')])))

No of positive in train : 12500
No of negative in train : 12500
No of positive in test : 12500
No of negative in test : 12500


In [8]:
full_dataset['review'].describe()

count                                                 50000
unique                                                49582
top       Loved today's show!!! It was a variety and not...
freq                                                      5
Name: review, dtype: object

In [9]:
full_dataset = full_dataset.drop_duplicates(subset=['review'])

In [10]:
full_dataset['review'].describe()

count                                                 49582
unique                                                49582
top       I went and saw this movie last night after bei...
freq                                                      1
Name: review, dtype: object

In [11]:
print("No of positive in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'train')])))
print("No of negative in train : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'train')])))
print("No of positive in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 1) & (full_dataset['set'] == 'test')])))
print("No of negative in test : "+str(len(full_dataset[(full_dataset['sentiment'] == 0) & (full_dataset['set'] == 'test')])))

No of positive in train : 12444
No of negative in train : 12337
No of positive in test : 12440
No of negative in test : 12361


In [12]:
train_dataset = full_dataset[(full_dataset['set'] == 'train')][['review','sentiment']]
test_dataset = full_dataset[(full_dataset['set'] == 'test')][['review','sentiment']]
test_dataset.head()

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [13]:
#Cleaning the dataset using Regex

import re
def preprocessing(texts):
  cleaned_text = []
  for text in texts:
    text = text.lower() #lower case the text
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                u"\U00002702-\U000027B0"
                                u"\U000024C2-\U0001F251"
                                "]+", flags=re.UNICODE)
    
    url_pattern = re.compile(r'https?://\S+|www\.\S+') #compile the urls/links patterns
    html_pattern = re.compile('<.*?>') #compile the html tafs
    text = emoji_pattern.sub(r'', text) #replace emojis
    text = url_pattern.sub(r'', text) #replace
    text = html_pattern.sub(r'', text)
    text = re.sub(r"[^\w\d'\s]+", ' ', text)#Skip words, numbers and space
    cleaned_text.append(text)

  return cleaned_text
  

In [ ]:
!pip install torch==1.8.0 torchtext==0.9.0 #compatibility

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy.data import Dataset, Example
from torchtext.legacy.data import BucketIterator


SEED = 42 #Variable from Pop Culture more info at https://medium.com/geekculture/the-story-behind-random-seed-42-in-machine-learning-b838c4ac290a
#It's just a seed to spawn random numbers

torch.manual_seed(SEED)
#Using deterministic algorithms
torch.backends.cudnn.deterministic = True # reproducibility of the training outcome
max_document_length = 300 #hyperparameter
  
#Text is the review                                                    #Tokenise using spacy
TEXT = data.Field(lower=True, include_lengths=True,  tokenize='spacy',     preprocessing=preprocessing,batch_first=True,  fix_length=max_document_length)
#Label is the sentiment
LABEL = data.Field(sequential=False, use_vocab=False)

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )



/usr/local/lib/python3.7/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [ ]:
#Pass the valid, test dataset
torch_valid_dataset, torch_test_dataset = DataFrameDataset(
    df=test_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
).split() 

In [ ]:
torch_valid_dataset

In [ ]:
torch_train_dataset = DataFrameDataset(
    df=train_dataset, 
    fields=(
        ('review', TEXT),
        ('sentiment', LABEL)
    )
)

In [ ]:
max_size = 30000 #hyperparameter vocab
TEXT.build_vocab(torch_train_dataset, max_size=max_size,vectors='fasttext.simple.300d')
vocab_size = len(TEXT.vocab)


.vector_cache/wiki.simple.vec: 293MB [00:08, 35.3MB/s]                           
100%|██████████| 111051/111051 [00:16<00:00, 6918.02it/s]


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[(' ', 785692), ('the', 326108), ('and', 161663), ('a', 160563), ('of', 144477), ('to', 134325), ('is', 109343), ('it', 92798), ('in', 92099), ('i', 81980), ('this', 72819), ('that', 72538), ("'s", 61640), (' br', 52238), ('was', 49935), ('as', 45797), ('for', 43582), ('with', 43503), ('movie', 42699), ('but', 41298)]


In [ ]:
BATCH_SIZE = 64 #hyperparameter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (torch_train_dataset, torch_valid_dataset, torch_test_dataset), 
    batch_size = BATCH_SIZE ,
    sort_key=lambda x: len(x.review),
    sort_within_batch=True,
    device = device)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class LR(nn.Module):
    def __init__(self, input_size, hidden_size,hidden_size2, num_classes):
        super(LR, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # 
        self.relu = nn.ReLU()
                              #input      #output
        self.fc2 = nn.Linear(hidden_size, hidden_size2)
                              #input       #output with classes to predict only 2 positive/negative 
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, text):
        text = text.float() # dense layer deals just with float type data
        x = self.fc1(text) #(m x n) with (n x p)
        x = self.relu(x)
        x = self.relu(self.fc2(x))
        
        preds = self.fc3(x) # crossentropyloss handles the softmax
        # preds = F.softmax(preds,1) # nn.softmax
        return preds

In [ ]:
#Set Hyperparamters

lr = 1e-4   #Learning rate
batch_size = 64   
dropout_keep_prob = 0.5
embedding_size = 300
max_document_length = 300 # each sentence has until 100 words
vocab_size = len(TEXT.vocab)
dev_size = 0.8 # split percentage to train\validation data
max_size = 30000 # maximum vocabulary size a.k.a unique words
seed = 42
num_classes = 2

num_epochs = 10   
hidden_size = 256  #nodes in the hidden layer 1
hidden_size1 = 300
hidden_size2 = 128
hidden_size3 = 64

to_train = True

model = LR(max_document_length, hidden_size,hidden_size2, num_classes)


In [ ]:
def accuracy(probs, target):
  winners = probs.argmax(dim=1)
  corrects = (winners == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy

In [ ]:
best_valid_loss = float('inf') #loss 
optimizer = torch.optim.Adam(model.parameters(), lr=lr) #optimizer using Adaptive with Numeriques(Adam)

loss_func = nn.CrossEntropyLoss() #Cross Entropy

for epoch in range(num_epochs):
  train_epoch_loss = 0 #start from 0
  train_epoch_acc = 0
  for batch in train_iterator:
      optimizer.zero_grad()
      # retrieve text and no. of words
      text, text_lengths = batch.review

      #feedforward
      predictions = model(text).squeeze(1) #feedfoward the input to predict
      
      
      loss = loss_func(predictions, batch.sentiment) #compute the difference from the actual/true output

      acc = accuracy(predictions, batch.sentiment) #current accuracy

      # perform backpropagation
      loss.backward()

      optimizer.step()  #Make an Increment to search for the lowest point on the graph

      train_epoch_loss += loss.item() 
      train_epoch_acc += acc.item()

  

  valid_epoch_loss = 0
  valid_epoch_acc = 0

  model.eval() #evaluate the model 

  with torch.no_grad():
      for batch in valid_iterator:
          text, text_lengths = batch.review

          predictions = model(text).squeeze(1)

          loss = loss_func(predictions, batch.sentiment)

          acc = accuracy(predictions, batch.sentiment)

          valid_epoch_loss += loss.item()
          valid_epoch_acc += acc.item()

   

  if valid_epoch_loss < best_valid_loss:
            best_valid_loss = valid_epoch_loss
            torch.save(model.state_dict(), 'saved_weights'+'_linear.pt')

  print(f'\tTrain Loss: {train_epoch_loss / len(train_iterator):.3f} | Train Acc: {train_epoch_acc  / len(train_iterator)* 100:.2f}%')
  print(f'\t Val. Loss: {valid_epoch_loss / len(valid_iterator):.3f} |  Val. Acc: {valid_epoch_acc / len(valid_iterator) * 100:.2f}%')


	Train Loss: 61.836 | Train Acc: 50.01%
	 Val. Loss: 36.990 |  Val. Acc: 49.74%
	Train Loss: 25.831 | Train Acc: 54.58%
	 Val. Loss: 26.427 |  Val. Acc: 49.91%
	Train Loss: 14.706 | Train Acc: 59.10%
	 Val. Loss: 21.743 |  Val. Acc: 49.73%
	Train Loss: 9.022 | Train Acc: 64.26%
	 Val. Loss: 18.248 |  Val. Acc: 49.82%
	Train Loss: 5.818 | Train Acc: 68.84%
	 Val. Loss: 16.205 |  Val. Acc: 50.12%
	Train Loss: 3.765 | Train Acc: 73.28%
	 Val. Loss: 14.612 |  Val. Acc: 49.78%
	Train Loss: 2.536 | Train Acc: 76.86%
	 Val. Loss: 13.637 |  Val. Acc: 49.87%
	Train Loss: 1.714 | Train Acc: 80.81%
	 Val. Loss: 12.871 |  Val. Acc: 50.04%
	Train Loss: 1.208 | Train Acc: 83.73%
	 Val. Loss: 12.231 |  Val. Acc: 50.06%
	Train Loss: 0.883 | Train Acc: 85.92%
	 Val. Loss: 11.872 |  Val. Acc: 50.06%
